In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Pinecone as PineconeVectorStore
from langchain.chains import RetrievalQA
from pinecone import Pinecone  
import yaml
import os
import zipfile


In [3]:
with open('conf.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['PINECONE_API_KEY'] = config['PINECONE_API_KEY']
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [4]:
zip_file_path = 'documentos.zip'
extracted_folder_path = 'docs'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

documents = []
for filename in os.listdir(extracted_folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(extracted_folder_path, filename)
        loader = PyMuPDFLoader(file_path)
        documents.extend(loader.load())

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.create_documents([doc.page_content for doc in documents])

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [7]:
index_name = "llm"
vector_store = PineconeVectorStore.from_documents(chunks, embedding=embeddings, index_name=index_name)

In [ ]:
query_1 = '''Responda apenas com base no input fornecido. Qual o número do processo que trata de Violação
de normas ambientais pela Empresa de Construção?'''

query_2 = 'Responda apenas com base no input fornecido. Qual foi a decisão no caso de fraude financeira?'

query_3 = 'Responda apenas com base no input fornecido. Quais foram as alegações no caso de negligência médica?'

query_4 = 'Responda apenas com base no input fornecido. Quais foram as alegações no caso de Número do Processo: 822162' #disputa contratual

In [9]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2)

In [10]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

In [11]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [12]:
print(chain)

verbose=False combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002471C4E03D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002471C4E0F90>, root_client=<openai.OpenAI object at 0x000002471C4

In [13]:
answer_1 = chain.invoke(query_1)
answer_2 = chain.invoke(query_2)
answer_3 = chain.invoke(query_3)
answer_4 = chain.invoke(query_4)

In [15]:
print('Pergunta: ',answer_1['query'])
print('Resultado: ',answer_1['result'],'\n')

print('Pergunta: ',answer_2['query'])
print('Resultado: ',answer_2['result'],'\n')

print('Pergunta: ',answer_3['query'])
print('Resultado: ',answer_3['result'],'\n')

print('Pergunta: ',answer_4['query'])
print('Resultado: ',answer_4['result'])

Pergunta:  Responda apenas com base no input fornecido. Qual o número do processo que trata de Violação
de normas ambientais pela Empresa de Construção?
Resultado:  O número do processo que trata da Violação de normas ambientais pela Empresa de Construção é 175543. 

Pergunta:  Responda apenas com base no input fornecido. Qual foi a decisão no caso de fraude financeira?
Resultado:  A decisão no caso de fraude financeira envolvendo João Almeida foi de uma sentença de dez anos de prisão e a restituição de $1,000,000 ao Banco Nacional. 

Pergunta:  Responda apenas com base no input fornecido. Quais foram as alegações no caso de negligência médica?
Resultado:  As alegações no caso de negligência médica foram de que o Dr. Pedro Sousa não realizou os procedimentos médicos com o cuidado necessário, resultando em complicações graves, incluindo uma cirurgia mal-sucedida que levou a infecções adicionais. Depoimentos de especialistas médicos apoiaram essas alegações. 

Pergunta:  Responda apenas 